In [3]:
import os
import glob
import time
import numpy as np
import pandas as pd
import seaborn as sn
import datetime as dt
import tensorflow as tf
import matplotlib.pyplot as plt

from os import path
from tensorflow import keras
from matplotlib.pyplot import *
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Dropout, Flatten, Convolution1D, UpSampling1D, MaxPooling1D
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, label_binarize
from sklearn import metrics
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score, multilabel_confusion_matrix

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [4]:
# full dataset
path = r'C:/Users/ADMIN/Desktop/code_folders/Capstone_Project/CIC-IDS/CSV-2023/MERGED_CSV'
all_files = glob.glob(path + "/*.csv")
li = []
for filename in all_files:
    df = pd.read_csv(filename, encoding='cp1252', index_col=None, header=0)
    li.append(df)
    print("Read Completed for ", filename)
df = pd.concat(li, axis=0, ignore_index=True)
df.describe()
df.head()
df.info()

Read Completed for  C:/Users/ADMIN/Desktop/code_folders/Capstone_Project/CIC-IDS/CSV-2023/MERGED_CSV\Merged01.csv
Read Completed for  C:/Users/ADMIN/Desktop/code_folders/Capstone_Project/CIC-IDS/CSV-2023/MERGED_CSV\Merged02.csv
Read Completed for  C:/Users/ADMIN/Desktop/code_folders/Capstone_Project/CIC-IDS/CSV-2023/MERGED_CSV\Merged03.csv
Read Completed for  C:/Users/ADMIN/Desktop/code_folders/Capstone_Project/CIC-IDS/CSV-2023/MERGED_CSV\Merged04.csv
Read Completed for  C:/Users/ADMIN/Desktop/code_folders/Capstone_Project/CIC-IDS/CSV-2023/MERGED_CSV\Merged05.csv
Read Completed for  C:/Users/ADMIN/Desktop/code_folders/Capstone_Project/CIC-IDS/CSV-2023/MERGED_CSV\Merged06.csv
Read Completed for  C:/Users/ADMIN/Desktop/code_folders/Capstone_Project/CIC-IDS/CSV-2023/MERGED_CSV\Merged07.csv
Read Completed for  C:/Users/ADMIN/Desktop/code_folders/Capstone_Project/CIC-IDS/CSV-2023/MERGED_CSV\Merged08.csv
Read Completed for  C:/Users/ADMIN/Desktop/code_folders/Capstone_Project/CIC-IDS/CSV-202

In [5]:
print(df.columns)
print(df["Label"].value_counts())
print(df.shape)

Index(['Header_Length', 'Protocol Type', 'Time_To_Live', 'Rate',
       'fin_flag_number', 'syn_flag_number', 'rst_flag_number',
       'psh_flag_number', 'ack_flag_number', 'ece_flag_number',
       'cwr_flag_number', 'ack_count', 'syn_count', 'fin_count', 'rst_count',
       'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP',
       'DHCP', 'ARP', 'ICMP', 'IGMP', 'IPv', 'LLC', 'Tot sum', 'Min', 'Max',
       'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Variance', 'Label'],
      dtype='object')
Label
DDOS-ICMP_FLOOD            6893259
DDOS-UDP_FLOOD             5181027
DDOS-TCP_FLOOD             4306086
DDOS-PSHACK_FLOOD          3920372
DDOS-SYN_FLOOD             3886130
DDOS-RSTFINFLOOD           3872808
DDOS-SYNONYMOUSIP_FLOOD    3445659
DOS-UDP_FLOOD              3177323
DOS-TCP_FLOOD              2558256
DOS-SYN_FLOOD              1942176
BENIGN                     1051373
MIRAI-GREETH_FLOOD          949381
MIRAI-UDPPLAIN              852695
MIRAI-GREIP_FLOOD   

In [6]:
DDoS = ['DDoS-ACK_Fragmentation',     
        'DDoS-UDP_Flood',       
        'DDoS-SlowLoris',               
        'DDoS-ICMP_Flood',        
        'DDoS-RSTFIN_Flood',    
        'DDoS-PSHACK_Flood',        
        'DDoS-HTTP_Flood',              
        'DDoS-UDP_Fragmentation',     
        'DDoS-TCP_Flood',  
        'DDoS-SYN_Flood',        
        'DDoS-SynonymousIP_Flood',    
        'DDoS-ICMP_Fragmentation']

DoS = ['DoS-UDP_Flood',    
       'DoS-TCP_Flood',    
       'DoS-SYN_Flood',    
       'DoS-HTTP_Flood']

Spoofing = ['MITM-ArpSpoofing',     
            'DNS_Spoofing']

Brute_force = ['DictionaryBruteForce']

Recon = ['Recon-HostDiscovery', 
         'Recon-OSScan',     
         'Recon-PortScan',               
         'Recon-PingSweep',         
         'VulnerabilityScan']

Web_based = ['SqlInjection',        
             'BrowserHijacking',         
             'CommandInjection',       
             'XSS',         
             'Backdoor_Malware',          
             'Uploading_Attack']

Mirai = ['Mirai-greeth_flood',     
         'Mirai-udpplain',     
         'Mirai-greip_flood']



def classify_attacks(data):
    data['Label'].replace(DDoS,'DDoS',inplace=True)
    data['Label'].replace(DoS,'DoS',inplace=True)
    data['Label'].replace(Spoofing,'Spoofing',inplace=True)
    data['Label'].replace(Brute_force,'Brute_Force',inplace=True)
    data['Label'].replace(Recon,'Recon',inplace=True)
    data['Label'].replace(Web_based,'Web_based',inplace=True)
    data['Label'].replace(Mirai,'Mirai',inplace=True)
classify_attacks(df)

print(df["Label"].value_counts())
print(df.shape)

Label
DDOS-ICMP_FLOOD            6893259
DDOS-UDP_FLOOD             5181027
DDOS-TCP_FLOOD             4306086
DDOS-PSHACK_FLOOD          3920372
DDOS-SYN_FLOOD             3886130
DDOS-RSTFINFLOOD           3872808
DDOS-SYNONYMOUSIP_FLOOD    3445659
DOS-UDP_FLOOD              3177323
DOS-TCP_FLOOD              2558256
DOS-SYN_FLOOD              1942176
BENIGN                     1051373
MIRAI-GREETH_FLOOD          949381
MIRAI-UDPPLAIN              852695
MIRAI-GREIP_FLOOD           719655
DDOS-ICMP_FRAGMENTATION     433157
VULNERABILITYSCAN           357583
MITM-ARPSPOOFING            294469
DDOS-UDP_FRAGMENTATION      274909
DDOS-ACK_FRAGMENTATION      272793
DNS_SPOOFING                171468
RECON-HOSTDISCOVERY         128677
RECON-OSSCAN                 93970
RECON-PORTSCAN               78730
DOS-HTTP_FLOOD               68799
DDOS-HTTP_FLOOD              27597
DDOS-SLOWLORIS               22400
DICTIONARYBRUTEFORCE         12522
BROWSERHIJACKING              5630
COMMANDINJECTI

In [7]:
data_clean = df.dropna().reset_index()
data_clean.drop_duplicates(keep='first', inplace = True)
data_clean['Label'].value_counts()
print("Read {} rows.".format(len(data_clean)))

data_clean.columns
data_clean = data_clean.dropna().reset_index()
labelencoder = LabelEncoder()
data_clean['Label'] = labelencoder.fit_transform(data_clean['Label'])
data_clean['Label'].value_counts()
print(data_clean.shape)
print(data_clean['Label'].value_counts())

data_np = data_clean.to_numpy(dtype="float32")
#drop inf values
data_np = data_np[~np.isinf(data_np).any(axis=1)]
X = data_np[:, :-1]
enc = OneHotEncoder()
Y = enc.fit_transform(data_np[:, -1].reshape(-1, 1)).toarray()

Read 45018564 rows.
(45018564, 42)
Label
6     6893203
14    5180997
13    4306058
8     3920337
12    3886105
9     3872787
11    3445630
21    3177280
20    2558232
19    1942159
1     1051313
22     949325
24     852632
23     719604
7      433115
32     357579
25     294451
15     274881
4      272767
17     171463
26     128676
27      93966
29      78729
18      68798
5       27597
10      22399
16      12522
2        5630
3        5168
30       5021
33       3705
0        3078
28       2161
31       1196
Name: count, dtype: int64


In [8]:
print(data_np.shape)

(45018243, 42)


In [9]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [10]:
# Split the data set into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, Y, test_size=0.25, random_state=33, shuffle=True)

_features = X.shape[1]
n_classes = Y.shape[1]

X_train = X_train.reshape(X_train.shape[0], _features, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], _features, 1).astype('float32')

In [11]:
# CNN model
# Convolutional Encoder
input_img = Input(shape=(_features, 1))

conv_1 = Convolution1D(256, 3, activation='relu')(input_img)
pool_1 = MaxPooling1D(3)(conv_1)

# Bottleneck
conv_2 = Convolution1D(128, 3, activation='relu')(pool_1)

# Decoder
up_1 = UpSampling1D(3)(conv_2)
conv_3 = Convolution1D(256, 3, activation='relu')(up_1)

In [12]:
# Classification
flatten = Flatten()(conv_3)

fc1 = Dense(256, activation='relu')(flatten)
do1 = Dropout(0.1)(fc1)

fc2 = Dense(128, activation='relu')(do1)
do2 = Dropout(0.1)(fc2)

fc3 = Dense(128, activation='relu')(do2)
do3 = Dropout(0.1)(fc3)

fc4 = Dense(n_classes, kernel_initializer='normal')(do3)

softmax = Dense(n_classes, activation='softmax', name='classification')(fc4)

model = Model(inputs=input_img, outputs=softmax)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 41, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 39, 256)        │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 13, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 11, 128)        │        98,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling1d (UpSampling1D)    │ (None, 33, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 31, 256)        │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 7936)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     2,031,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 34)             │         4,386 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classification (Dense)          │ (None, 34)             │         1,190 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,284,872 (8.72 MB)

 Trainable params: 2,284,872 (8.72 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy',optimizer=opt, metrics=['accuracy'])
early_stop_callback = keras.callbacks.EarlyStopping(monitor='loss', patience=5)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                  patience=3, min_lr=0.0001)

In [14]:
start_fit = time.time()
history = model.fit(X_train, Y_train,
                              batch_size=32,
                              epochs=30,
                              verbose=True,
                              validation_data=(X_test, Y_test))    

end_fit = time.time()
fit_time = end_fit - start_fit
print("fit time: {:.2f} seconds".format(fit_time))  

Epoch 1/30
  10588/1055116 ━━━━━━━━━━━━━━━━━━━━ 2:35:48 9ms/step - accuracy: 0.6918 - loss: 0.7302

KeyboardInterrupt: 

In [ ]:
fitting_time_np = np.array([['Start time', 'End time', 'Fitting time'],
                            [start_fit, end_fit, fit_time]])

fitting_time_np

In [ ]:
# np.save('D:/results/CICIoT2023/History/[CICIoT2023] [CNN] Multiclass classification history.npy',history.history)
# np.save('D:/results/CICIoT2023/History/[CICIoT2023] [CNN] Multiclass classification Fitting time.npy',fitting_time_np)

In [ ]:
# Plot for training and validation loss
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
start_by_epoch = 1
epochs = range(start_by_epoch, len(loss_values) + 1)

In [ ]:
# Exploratory data analysis
plt.plot(epochs, acc[start_by_epoch-1:], label='Training accuracy', marker='x', markersize=2.5, mew=0.4, linewidth=0.8)
plt.plot(epochs, val_acc[start_by_epoch-1:], label='Validation accuracy', marker='x', markersize=2.5, mew=0.4, linewidth=0.8)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training', 'Validation'], loc='lower right')
plt.grid()
ax = plt.gca()
plt.tight_layout()
ax.spines['bottom'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.ylim(0.97,1)
plt.savefig('D:/results/CICIoT2023/figures/[CICIoT2023] [CNN] Multiclass classification accuracy.eps', format='eps', dpi=1200)
plt.show()
plt.clf()

In [ ]:
plt.plot(epochs, loss_values[start_by_epoch-1:], label='Training Loss', marker='x', markersize=2.5, mew=0.4, linewidth=0.8)
plt.plot(epochs, val_loss_values[start_by_epoch-1:], label='Validation Loss', marker='x', markersize=2.5, mew=0.4, linewidth=0.8)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.grid()
ax = plt.gca()
plt.tight_layout()
ax.spines['bottom'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.ylim(0,0.07)
plt.savefig('D:/results/CICIoT2023/figures/[CICIoT2023] [CNN] Multiclass classification loss.eps', format='eps', dpi=1200)
plt.show()
plt.clf()

In [ ]:
start_time = time.time()
pred = model.predict(X_test)
end_time = time.time()

inference_time = end_time - start_time
print("Inference time: {:.2f} seconds".format(inference_time))

inference_time_np = np.array([['Start time', 'End time', 'Inference time'],
                            [start_time, end_time, inference_time]])

In [ ]:
# np.save('D:/results/CICIoT2023/history/[CICIoT2023] [CNN] Multiclass classification pred.npy',pred)
# np.save('D:/results/CICIoT2023/history/[CICIoT2023] [CNN] Multiclass classification Inference time.npy',inference_time_np)

In [ ]:
# Convert predictions classes to one hot vectors 
pred = np.argmax(pred,axis=1)
# Convert validation observations to one hot vectors
y_test = Y_test.argmax(axis=1)

print(pred.shape)
print(y_test.shape)

score = metrics.accuracy_score(y_test, pred)
rscore = recall_score(y_test, pred, average='weighted')
ascore = precision_score(y_test, pred, average='weighted')
ascore_macro = precision_score(y_test, pred, average='macro')
f1score= f1_score(y_test, pred, average='weighted') #F1 = 2 * (precision * recall) / (precision + recall) for manual

print("Validation score: {}".format(score))
print("Recall score: {}".format(rscore))
print("Precision score: {}".format(ascore))
print("Precision score macro: {}".format(ascore_macro))
print("F1 Measure score: {}".format(f1score))